In [2]:
!pip install torchinfo

In [3]:
import torch
import cv2
import time

import torch.nn as nn
import numpy as np

from torchvision import transforms
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from torchinfo import summary

import math

In [4]:
def video_from_frames(filename, frames):
    vw = cv2.VideoWriter("filename", cv2.VideoWriter_fourcc(*"MJPG"), 15, (640,480))

    for frame in video:
        vw.write(frame)

    vw.release()

#Phase Correlation

In [ ]:
def test_setting_phase_corelation(video_cap, dp_threshold):

    u_frame = None
    u_frame_value = None
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (10, 50)
    fontScale = 2
    color = (255, 0, 0)
    thickness = 2
    video = []
    ess = 0
    time = 1
    
    while (video_cap.isOpened()):
        ret, frame = video_cap.read()
        
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            if u_frame is None:
                u_frame = frame
                continue
            
            dp = cv2.phaseCorrelate(np.float32(u_frame), np.float32(frame))
            (x,y),z = dp
            r = math.sqrt(x*x + y*y)

            if r < dp_threshold:
                frame = cv2.putText(frame, f"OK", org, font, fontScale, color, thickness, cv2.LINE_AA)
            else:
                u_frame = frame
                frame = cv2.putText(frame, f"NOTOK", org, font, fontScale, color, thickness, cv2.LINE_AA)
                ess += 1
            
            video.append(frame)
            time += 1
        else:
            break

    print(ess/time)
    
    video_cap.release()
    return video

In [ ]:
cap = cv2.VideoCapture('/content/output.mp4')

In [ ]:
video = test_setting(cap, 20)

0.35342465753424657


In [ ]:
video_from_frames("output.avi", video)

# Neural Network

In [ ]:
model = torch.load('/content/drive/MyDrive/Models/MobileNetV3_small_torch_h')
model.eval()

In [ ]:
image_net_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ]
    )

In [6]:
def split_network(model, index):
    network_1 = nn.Sequential(model[0][:index])
    network_2 = nn.Sequential(*list(model[0][index:]) + list(model[1:]))
    return network_1,network_2

In [ ]:
class DefaultComparator():
    def compare(self, n1, n2):
        raise NotImplementedError

class MeanSquareComparator(DefaultComparator):
    def __init__(self, threshold):
        self.threshold = threshold
    
    def compare(self, o1, o2):
        return torch.mean((o1-o2)**2) < self.threshold, torch.mean((o1-o2)**2)

class ZBYCHUComparator(DefaultComparator):
    def __init__(self, threshold):
        self.threshold = threshold
    
    def compare(self, o1, o2):
        return torch.mean((o1-o2)**2) < (torch.mean(o1**2) * self.threshold), torch.mean((o1-o2)**2)

In [ ]:
def test_setting_neural_netowrk(video_cap, network_1, comparator):

    u_frame = None
    u_frame_value = None
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (10, 50)
    fontScale = 2
    color = (255, 0, 0)
    thickness = 2


    time = 0
    ess = 0
    video = []
    
    while (video_cap.isOpened()):
        ret, frame = video_cap.read()
        if ret == True:
            
            if u_frame is None:
                u_frame = frame
                u_frame_value = network_1(image_net_transforms(frame).view(1,3,224,224))
                continue

            frame_value = network_1(image_net_transforms(frame).view(1,3,224,224))
            is_ok, value = comparator.compare(frame_value, u_frame_value)
            
            if is_ok:
                frame = cv2.putText(frame, f"OK", org, font, fontScale, color, thickness, cv2.LINE_AA)
            else:
                ess += 1
                u_frame = frame
                u_frame_value = frame_value
                frame = cv2.putText(frame, f"NOTOK", org, font, fontScale, color, thickness, cv2.LINE_AA)
            
            video.append(frame)
            time += 1
        else:
            break
    print(ess/time)
    
    video_cap.release()
    return video

In [ ]:
comparator = ZBYCHUComparator(0.8)
n1,n2 = split_network(model, 3)
cap = cv2.VideoCapture('/content/output.mp4')

In [ ]:
video = test_setting(cap, n1, comparator)

0.41838134430727025


In [ ]:
video_from_frames("output.avi", video)

In [10]:
n1,n2 = split_network(model, 3)

In [11]:
torch.save(n1, "MobilenetV3_torch_first")

In [12]:
torch.save(n2, "MobilenetV3_torch_second")